In [1]:
import os
import time
import csv
import sys
import yaml
import numpy as np
import pandas as pd
from src.util_bckup import ExeDataset, write_pred
from src.model import MalConv
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import pickle
import time

use_gpu = True
use_cuda = torch.cuda.is_available()

malconv = torch.load('/home/user/Desktop/Retraining_Malconv/checkpoint/Retrain_all_samples_sd_850.model', map_location=torch.device('cuda') if use_gpu else 'cpu')
malconv = malconv.cuda() if use_gpu else malconv
bce_loss = nn.BCEWithLogitsLoss().cuda() if use_gpu else nn.BCEWithLogitsLoss()

print("Loading MalConv model successful")


/home/user/.local/lib/python3.10/site-packages/numpy/lib/scimath.py:46: RuntimeWarning: divide by zero encountered in log
  _ln2 = nx.log(2.0)
/tmp/ipykernel_3674/697076042.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub fo

Loading MalConv model successful


In [2]:
data_path = '/home/user/Desktop/Obfus_malware/SelfObfuscate/kkrunchy/Krunchy2Test/'

validloader = DataLoader(ExeDataset(data_path),
                         batch_size=1, shuffle=False, num_workers=0)

In [3]:

def loss_function_gradient(malconv, original_x, adv_x, use_cuda=False):
    y = malconv.embedd_and_forward(adv_x)
    y = nn.Sigmoid()(y)
    
    malware_class = torch.ones(y.shape)
    if use_cuda:
        malware_class = malware_class.cuda()
    loss = torch.nn.functional.binary_cross_entropy(y, malware_class)
    g = torch.autograd.grad(loss, adv_x)[0]
    g = torch.transpose(g, -1, -2)[0]
    return g	

def optimization_solver(gradient_f, index_to_consider, x_init):
    device = gradient_f.device  # Get the device of gradient_f (it should be on the same device as your model)

    # Create a zero tensor on the same device as gradient_f
    zero_tensor = torch.zeros(gradient_f.shape, device=device)

    # Compare if gradient_f is zero using tensors on the same device
    g = gradient_f / torch.norm(gradient_f) if not torch.equal(gradient_f, zero_tensor) else torch.zeros(gradient_f.shape, device=device)
    
    epsilon = 100
    gradient_result = (epsilon * g).transpose(0,1)
    x_init_updated = x_init.clone()  # Create a clone of x_init to avoid in-place operation
    x_init_updated[0, :, index_to_consider] = x_init_updated[0, :, index_to_consider] + gradient_result[:, index_to_consider]
    
    return x_init_updated


In [4]:
import torch.multiprocessing as mp
import gc
mp.set_start_method('spawn', force=True)
torch.multiprocessing.set_sharing_strategy('file_system')

In [5]:
df = pd.DataFrame(columns=['val_batch_data', 'length', 'init_prob','iteration','progress'])
counter = 0
success = 0
for _, val_batch_data in enumerate(validloader):
    counter +=1
    if counter%100==0:
        print(f"Completed: {counter}")
        print(success)
        torch.cuda.empty_cache()
    exe_input = val_batch_data[0].cuda() if use_gpu else val_batch_data[0]
    data = exe_input[0].cpu().numpy()
    length = data[-1]
    
    data = data[:length]
    data = np.concatenate([data, np.random.randint(0, 256, 2000000 - length)])

    embed = malconv.embed
    sigmoid = nn.Sigmoid()

    x0 = torch.from_numpy(np.array([data])).long().cuda() if use_gpu else torch.from_numpy(np.array([data])).long()
    x0 = Variable(x0.long(), requires_grad=False)
    pred, x_init = malconv(x0)
    initial_prob = sigmoid(pred).cpu().data.numpy()[0][0]

    if (length+12288)>2000000 or initial_prob<0.5:
        continue
    
    index_to_consider = list(range(length, length+12288))

    for i in range(50):
        gradient_f = loss_function_gradient(malconv, x0, x_init, use_cuda=True)
        x_init = optimization_solver(gradient_f, index_to_consider, x_init)
        progress = sigmoid(malconv.embedd_and_forward(x_init)).cpu().data.numpy()[0][0]
        if progress<0.5:
            success+=1
            print(progress)
            break
    # # Data to append as a dictionary
    # new_data = {'val_batch_data': val_batch_data, 'length': length, 'init_prob': initial_prob, 'iteration': i, 'progress':progress}
    # new_data_df = pd.DataFrame([new_data])
    # # Use pd.concat to append the new row
    # df = pd.concat([df, new_data_df], ignore_index=True)
    # if counter%2000==0:
    #     df.to_csv('Malconv_Append_8192_UPX.csv', mode='a', header=False, index=False) 
print(f"Success: {success}/{counter}")
    

0.26520413
0.0025087055
0.00052700477
0.09160258
0.007390596
0.17498253
0.026405536
0.0127522
0.46669576
0.1391384
0.12490935
0.017273111
0.2848499
0.2963913
0.00057744334
0.21532091
0.21571058
0.006445812
0.4991257
0.008492495
0.056381524
0.00097203994
0.0006991666
0.13907446
9.212619e-05
0.3552888
0.055830378
0.2041733
0.000101375466
0.0031186757
0.0009501834
0.390898
0.4976469
Completed: 100
33
0.34258297
0.038337994
0.42450407
0.041080132
0.077047154
0.37930173
0.16473146
2.6829838e-10
0.010860658
0.053593572
0.03421602
0.0013038135
0.40623215
0.009243871
0.26622164
0.48885188
0.0031843903
0.0059739444
0.24712932
0.41407722
0.07674333
0.052222952
0.018262707
0.069260955
0.0053405603
0.1641423
0.42998666
0.01016225
0.016288038
Completed: 200
62
0.18121374
3.8603484e-05
0.47400913
0.03915647
0.31449315
0.008702559
0.07469246
0.05410907
0.4372056
0.017277824
0.30895934
0.058200948
0.0023041803
0.20133588
0.0036354447
0.20893581
0.0077100215
0.4415329
0.004698249
0.008740199
0.00016658

In [6]:
df = pd.DataFrame(columns=['val_batch_data', 'length', 'init_prob','iteration','progress'])
counter = 0
success = 0
for _, val_batch_data in enumerate(validloader):
    counter +=1
    if counter%100==0:
        print(f"Completed: {counter}")
        print(success)
        torch.cuda.empty_cache()
    exe_input = val_batch_data[0].cuda() if use_gpu else val_batch_data[0]
    data = exe_input[0].cpu().numpy()
    length = data[-1]
    
    data = data[:length]
    data = np.concatenate([data, np.random.randint(0, 256, 2000000 - length)])

    embed = malconv.embed
    sigmoid = nn.Sigmoid()

    x0 = torch.from_numpy(np.array([data])).long().cuda() if use_gpu else torch.from_numpy(np.array([data])).long()
    x0 = Variable(x0.long(), requires_grad=False)
    pred, x_init = malconv(x0)
    initial_prob = sigmoid(pred).cpu().data.numpy()[0][0]

    if (length+16384)>2000000 or initial_prob<0.5:
        continue
    
    index_to_consider = list(range(length, length+16384))

    for i in range(50):
        gradient_f = loss_function_gradient(malconv, x0, x_init, use_cuda=True)
        x_init = optimization_solver(gradient_f, index_to_consider, x_init)
        progress = sigmoid(malconv.embedd_and_forward(x_init)).cpu().data.numpy()[0][0]
        if progress<0.5:
            success+=1
            print(progress)
            break
    # # Data to append as a dictionary
    # new_data = {'val_batch_data': val_batch_data, 'length': length, 'init_prob': initial_prob, 'iteration': i, 'progress':progress}
    # new_data_df = pd.DataFrame([new_data])
    # # Use pd.concat to append the new row
    # df = pd.concat([df, new_data_df], ignore_index=True)
    # if counter%2000==0:
    #     df.to_csv('Malconv_Append_8192_UPX.csv', mode='a', header=False, index=False) 
print(f"Success: {success}/{counter}")
    

0.00045889892
0.2015936
0.034243442
0.0009742708
3.9909393e-13
0.31631154
0.23911425
0.23246975
0.0002885825
0.45339277
0.027271353
0.009769625
0.49780858
0.0040878113
0.08150855
0.005395393
0.16248344
0.009969822
0.2237065
0.0029065649
0.22146177
0.06181414
0.06648666
0.0073691388
0.26297143
0.030581977
1.4968015e-05
0.02189756
0.0023267218
0.041798875
0.47150555
0.26093668
0.00067289744
0.30873093
0.27200788
0.023769775
0.013928709
0.20790279
Completed: 100
38
0.0177545
0.018215215
0.3751809
0.02614408
0.31759238
0.3513913
0.00091767404
0.031688333
0.24738121
0.028625447
0.10223094
0.002114037
2.8618279e-05
0.14198776
0.00013008167
0.0008765548
0.00012503265
0.006585537
0.38115698
0.00022222007
0.051207695
9.317848e-06
0.055318628
0.07028031
0.009561404
0.4576878
3.126577e-06
0.3257836
0.021890715
0.000107900574
0.020903833
0.24616942
0.40892154
0.04194539
4.3518926e-06
0.182431
Completed: 200
74
1.9909605e-05
0.06304595
0.06673308
0.091497615
0.0056209746
0.01876472
0.00014805135
0.

In [7]:
df = pd.DataFrame(columns=['val_batch_data', 'length', 'init_prob','iteration','progress'])
counter = 0
success = 0
for _, val_batch_data in enumerate(validloader):
    counter +=1
    if counter%100==0:
        print(f"Completed: {counter}")
        print(success)
        torch.cuda.empty_cache()
    exe_input = val_batch_data[0].cuda() if use_gpu else val_batch_data[0]
    data = exe_input[0].cpu().numpy()
    length = data[-1]
    
    data = data[:length]
    data = np.concatenate([data, np.random.randint(0, 256, 2000000 - length)])

    embed = malconv.embed
    sigmoid = nn.Sigmoid()

    x0 = torch.from_numpy(np.array([data])).long().cuda() if use_gpu else torch.from_numpy(np.array([data])).long()
    x0 = Variable(x0.long(), requires_grad=False)
    pred, x_init = malconv(x0)
    initial_prob = sigmoid(pred).cpu().data.numpy()[0][0]

    if (length+20480)>2000000 or initial_prob<0.5:
        continue
    
    index_to_consider = list(range(length, length+20480))

    for i in range(50):
        gradient_f = loss_function_gradient(malconv, x0, x_init, use_cuda=True)
        x_init = optimization_solver(gradient_f, index_to_consider, x_init)
        progress = sigmoid(malconv.embedd_and_forward(x_init)).cpu().data.numpy()[0][0]
        if progress<0.5:
            success+=1
            print(progress)
            break
    # # Data to append as a dictionary
    # new_data = {'val_batch_data': val_batch_data, 'length': length, 'init_prob': initial_prob, 'iteration': i, 'progress':progress}
    # new_data_df = pd.DataFrame([new_data])
    # # Use pd.concat to append the new row
    # df = pd.concat([df, new_data_df], ignore_index=True)
    # if counter%2000==0:
    #     df.to_csv('Malconv_Append_8192_UPX.csv', mode='a', header=False, index=False) 
print(f"Success: {success}/{counter}")
    

0.2653897
0.00228748
0.25541744
0.019611275
0.0014311131
0.0031971054
1.5298172e-05
0.16741146
2.9569347e-07
0.012345852
0.030031428
0.06808021
0.080317125
0.41246226
0.17908446
0.30854085
0.2643371
0.0012028579
0.3486321
0.3662995
0.00031567217
0.00030514348
0.08064873
0.04497439
0.027070796
0.005205364
2.2703975e-07
0.3164883
2.2390912e-08
0.00023516521
0.00018589295
0.32837498
0.26366556
0.11777841
0.15182659
0.000403938
0.0010187038
0.17705372
0.026961803
0.25558296
0.20945199
Completed: 100
41
5.3127323e-20
0.34075293
0.0030139291
0.23558079
0.02241996
0.15660515
0.02366203
0.28704214
0.07899822
0.13068636
0.032158025
0.000185453
0.11760457
0.32936677
0.3975488
1.7966908e-05
0.21676522
0.473666
0.10488015
0.3375036
0.4825865
0.000539109
0.052398942
0.00061981444
0.31563193
0.048242427
0.31677786
0.035095137
2.2862965e-05
0.06446601
0.31968302
0.00033547357
0.0031274105
0.038021408
Completed: 200
75
0.010100103
0.13625993
0.19963185
0.49841267
0.21660428
0.014148097
0.42087695
0.09

In [8]:
# import concurrent.futures
# from tqdm import tqdm

# # Create an empty DataFrame to store results
# df = pd.DataFrame(columns=['val_batch_data', 'length', 'init_prob', 'iteration', 'progress'])
# # Helper functions: Assuming loss_function_gradient, optimization_solver, etc., are defined elsewhere.

# def process_batch(val_batch_data, malconv, use_gpu):
#     # This function processes a single batch and returns the results to append to the dataframe
#     exe_input = val_batch_data[0].cuda() if use_gpu else val_batch_data[0]
#     data = exe_input[0].cpu().numpy()
#     length = data[-1]

#     data = data[:length]
#     data = np.concatenate([data, np.random.randint(0, 256, 2000000 - length)])

#     sigmoid = nn.Sigmoid()

#     x0 = torch.from_numpy(np.array([data])).long().cuda() if use_gpu else torch.from_numpy(np.array([data])).long()
#     x0 = Variable(x0.long(), requires_grad=False)
#     pred, x_init = malconv(x0)
#     initial_prob = sigmoid(pred).cpu().data.numpy()[0][0]

#     if (length+8192) > 2000000 or initial_prob < 0.5:
#         return None  # Skip if conditions are not met

#     index_to_consider = list(range(length, length+8192))

#     for i in range(50):
#         gradient_f = loss_function_gradient(malconv, x0, x_init, use_cuda=True)
#         x_init = optimization_solver(gradient_f, index_to_consider, x_init)
#         progress = sigmoid(malconv.embedd_and_forward(x_init)).cpu().data.numpy()[0][0]
#         if progress < 0.5:
#             print(progress)
#             break

#     # Return results as a dictionary
#     return {
#         'val_batch_data': val_batch_data,
#         'length': length,
#         'init_prob': initial_prob,
#         'iteration': i,
#         'progress': progress
#     }

# def append_to_df(results):
#     # Append data to DataFrame (if result is not None)
#     if results:
#         new_data_df = pd.DataFrame([results])
#         return new_data_df
#     return pd.DataFrame()

# # Assuming validloader and malconv are already defined
# use_gpu = torch.cuda.is_available()

# # Parallelize batch processing using concurrent.futures.ThreadPoolExecutor
# counter = 0
# with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#     futures = []
#     for val_batch_data in validloader:
#         futures.append(executor.submit(process_batch, val_batch_data, malconv, use_gpu))

#     for future in concurrent.futures.as_completed(futures):
#         counter +=1
#         if counter%100 == 0:
#             print(f" Completed :{counter}")
#         result = future.result()
#         new_data_df = append_to_df(result)
#         df = pd.concat([df, new_data_df], ignore_index=True)

# print(df)
